In [3]:
import wikipediaapi
wiki = wikipediaapi.Wikipedia('en')

## Try Biomedical Category First

In [3]:
def print_categorymembers(categorymembers, level=0, max_level=2):
        for c in categorymembers.values():
            print("%s: %s (ns: %d)" % ("*" * (level + 1), c.title, c.ns))
            if c.ns == wikipediaapi.Namespace.CATEGORY and level <= max_level:
                print_categorymembers(c.categorymembers, level + 1)


cat = wiki.page("Category:Biomedical")
print("Category members: Category:Biomedical")
print_categorymembers(cat.categorymembers)

Category members: Category:Biomedical


## Find Proper Category for Biomedical Field
### From biomedical engineering introduction page

In [5]:
be = wiki.page('Biomedical engineering')
be.exists()

True

In [7]:
be = wiki.page('Biomedical engineering')

def print_categories(page):
        categories = page.categories
        for title in sorted(categories.keys()):
            print(title, categories[title])


print("Categories")
print_categories(be)

Categories
Category:All articles needing additional references Category:All articles needing additional references (id: ??, ns: 14)
Category:Articles needing additional references from August 2017 Category:Articles needing additional references from August 2017 (id: ??, ns: 14)
Category:Articles needing additional references from July 2017 Category:Articles needing additional references from July 2017 (id: ??, ns: 14)
Category:Articles needing additional references from September 2016 Category:Articles needing additional references from September 2016 (id: ??, ns: 14)
Category:Articles with Curlie links Category:Articles with Curlie links (id: ??, ns: 14)
Category:Biological engineering Category:Biological engineering (id: ??, ns: 14)
Category:Biomedical engineering Category:Biomedical engineering (id: ??, ns: 14)
Category:Webarchive template wayback links Category:Webarchive template wayback links (id: ??, ns: 14)
Category:Wikipedia articles with BNF identifiers Category:Wikipedia art

### Find all articles in Biomedical engineering

In [22]:
def print_categorymembers(categorymembers, level=0, max_level=2):
        for c in categorymembers.values():
            print("%s: %s (ns: %d)" % ("*" * (level + 1), c.title, c.ns))
            if c.ns == wikipediaapi.Namespace.CATEGORY and level <= max_level:
                print_categorymembers(c.categorymembers, level + 1)


cat = wiki.page("Category:Biomedical engineering")
print("Category members: Category:Biomedical engineering")
#print_categorymembers(cat.categorymembers)

Category members: Category:Biomedical engineering


### Add relationship by the links between pages

In [21]:
be = wiki.page('Biomedical engineering')

def print_links(page):
        links = page.links
        for title in sorted(links.keys()):
            print("%s: %s" % (title, links[title]))

#print_links(be)

In [12]:
test = wiki.page('Biomedical Equipment Technician')
test.title

'Biomedical Equipment Technician'

## Interface with mongoDB

In [2]:
from pymongo import MongoClient
client = MongoClient()
client = MongoClient('.amazonaws.com', 27017)
db = client.KG

In [71]:
db.categories.find_one({'title':"Category:Biomedical engineering in fiction"})

{'_id': ObjectId('5aba9fc532736325267ad2c4'),
 'level': 1,
 'title': 'Category:Biomedical engineering in fiction',
 'type': 'sub_category'}

In [1]:
# Search all categories that is a member of main category: Biochemical Engineering
def get_allsubcategories(cat, level = 0, max_level = 2):
    for c in cat.categorymembers.values():
        if c.ns == wikipediaapi.Namespace.CATEGORY and level <= max_level:
            if db.categories.find({'title':c.title}).count() == 0:
                db.categories.insert_one({'level':level+1, 'type':'sub_category','title':c.title,'parent':cat.title})
                get_allsubcategories(c,level+1)
            else:
                minlevel = min(level+1,db.categories.find_one({'title':c.title})['level'])
                db.categories.update_one({'title':c.title},{'$set':{'level':minlevel}})

In [4]:
name = "Category:Biomedical engineering"
cat = wiki.page(name)
db.categories.insert_one({'level':0, 'type':'main_category','title':name})
get_allsubcategories(cat)

In [6]:
#For all categories, get their pages
def get_allrelatednodes():
    for cat in db.categories.find({}):
        pages = wiki.page(cat['title']).categorymembers.values()
        for page in pages:
            if page.ns != wikipediaapi.Namespace.CATEGORY:
                if db.pages.find({'title':page.title}).count() == 0:
                    links = []
                    for link in page.links.keys():
                        links.append(link)
                    db.pages.insert_one({'title':page.title,'category':[cat['title']],'links':links})
                else:
                    newcat = db.pages.find_one({'title':page.title})['category']
                    if cat['title'] not in newcat:
                        newcat.append(cat['title'])
                    db.pages.update_one({'title':page.title},{'$set':{'category':newcat}})
                    
get_allrelatednodes()

In [14]:
db.cat_edges.remove({})

/home/ec2-user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  """Entry point for launching an IPython kernel.


{'n': 46, 'ok': 1.0}

In [20]:
#Link the pages and categories together
def get_alledges():
    #get_allcatedges()
    get_allpageedges()
    
def get_allcatedges():
    cats = db.categories.find({})
    for cat in cats:
        if cat['level'] > 0:
            db.cat_edges.insert_one({'Parent':db.categories.find_one({'title':cat['parent']}),'Child':cat,'content':'added via'})

def get_allpageedges():
    pages = db.pages.find({})
    for page in pages:
        for linktitle in page['links']:
            linkpage = db.pages.find_one({'title':linktitle})
            if db.page_edges.find({'Source':page,'Link':linkpage}).count() == 0:
                db.page_edges.insert_one({'Source':page,'Link':linkpage,'content':'has link to'})
            else:
                db.page_edges.update_one({'Source':page,'Link':linkpage},{'$set':{'content':'linked with each other'}})

get_alledges()